In [2]:
from sqlalchemy import create_engine, Column, Integer, String, Date, DateTime, Float, BigInteger, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
engine = create_engine("sqlite://", echo=True)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

/var/folders/hq/659b4zd16vscxjxb3lxdr18c0000gp/T/ipykernel_83383/1111253954.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
from sqlalchemy import create_engine, text
import re

# Connection details
USERNAME = 'root'
PASSWORD = 'TSUtigers123'
SERVER = 'localhost'
DATABASE = 'cdm'

# SQLAlchemy connection string for MySQL using mysqlconnector
connection_string = f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}'

# Creating the engine for MySQL
engine = create_engine(connection_string, echo=True)

In [95]:

sql_file_path = './OMOPCDM_sql_server_5.4_ddl.sql'

cleaned_sql_commands = []
with open(sql_file_path, 'r') as sql_file:
    for line in sql_file:
        if not line.strip().startswith(('HINT', '--')):
            line = re.sub(r'--.*$', '', line)
            line = line.replace('@cdmDatabaseSchema.', '')
            cleaned_sql_commands.append(line)

sql_commands = ''.join(cleaned_sql_commands)
commands = re.split(r';\s*(?=\n)', sql_commands)

with engine.begin() as conn:
    for command in commands:
        if command.strip():  # Avoid executing empty or whitespace-only commands
            conn.execute(text(command))


2024-04-18 22:58:07,224 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-18 22:58:07,225 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:58:07,228 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-18 22:58:07,228 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:58:07,229 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-18 22:58:07,229 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:58:07,231 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 22:58:07,231 INFO sqlalchemy.engine.Engine 
CREATE TABLE PERSON (
			person_id integer NOT NULL,
			gender_concept_id integer NOT NULL,
			year_of_birth integer NOT NULL,
			month_of_birth integer NULL,
			day_of_birth integer NULL,
			birth_datetime datetime NULL,
			race_concept_id integer NOT NULL,
			ethnicity_concept_id integer NOT NULL,
			location_id integer NULL,
			provider_id integer NULL,
			care_site_id integer NULL,
			person_source_value varchar(50) NULL,
			gender_sou

In [98]:
sql_file_path = './OMOPCDM_sql_server_5.4_primary_keys.sql'

# Read SQL commands from the file and replace the placeholder
with open(sql_file_path, 'r') as file:
    sql_commands = file.read().replace('@cdmDatabaseSchema.', '')

# Split the modified SQL commands into individual commands
# Assuming each command ends with a ';' and possibly a newline for readability
commands = sql_commands.split(';\n')

# Execute each command
with engine.begin() as connection:
    for command in commands:
        command = command.strip()  # Clean whitespace
        if command:  # Check if command is not empty
            connection.execute(text(command))

2024-04-18 22:59:29,272 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 22:59:29,275 INFO sqlalchemy.engine.Engine ALTER TABLE PERSON ADD CONSTRAINT xpk_PERSON PRIMARY KEY NONCLUSTERED (person_id)
2024-04-18 22:59:29,276 INFO sqlalchemy.engine.Engine [generated in 0.00119s] {}
2024-04-18 22:59:29,341 INFO sqlalchemy.engine.Engine ALTER TABLE OBSERVATION_PERIOD ADD CONSTRAINT xpk_OBSERVATION_PERIOD PRIMARY KEY NONCLUSTERED (observation_period_id)
2024-04-18 22:59:29,342 INFO sqlalchemy.engine.Engine [generated in 0.00073s] {}
2024-04-18 22:59:29,355 INFO sqlalchemy.engine.Engine ALTER TABLE VISIT_OCCURRENCE ADD CONSTRAINT xpk_VISIT_OCCURRENCE PRIMARY KEY NONCLUSTERED (visit_occurrence_id)
2024-04-18 22:59:29,356 INFO sqlalchemy.engine.Engine [generated in 0.00079s] {}
2024-04-18 22:59:29,369 INFO sqlalchemy.engine.Engine ALTER TABLE VISIT_DETAIL ADD CONSTRAINT xpk_VISIT_DETAIL PRIMARY KEY NONCLUSTERED (visit_detail_id)
2024-04-18 22:59:29,370 INFO sqlalchemy.engine.Engine [gen

In [99]:
from sqlalchemy.engine.reflection import Inspector

inspector = Inspector.from_engine(engine)

# Retrieve and print all table names using the Inspector
table_names = inspector.get_table_names()
print("Tables in the database:")
for table_name in table_names:
    print(table_name)

2024-04-18 22:59:34,455 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 22:59:34,456 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `cdm`
2024-04-18 22:59:34,456 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:59:34,485 INFO sqlalchemy.engine.Engine ROLLBACK
Tables in the database:
CARE_SITE
CDM_SOURCE
COHORT
COHORT_DEFINITION
CONCEPT
CONCEPT_ANCESTOR
CONCEPT_CLASS
CONCEPT_RELATIONSHIP
CONCEPT_SYNONYM
CONDITION_ERA
CONDITION_OCCURRENCE
COST
DEATH
DEVICE_EXPOSURE
DOMAIN
DOSE_ERA
DRUG_ERA
DRUG_EXPOSURE
DRUG_STRENGTH
EPISODE
EPISODE_EVENT
FACT_RELATIONSHIP
LOCATION
MEASUREMENT
METADATA
NOTE
NOTE_NLP
OBSERVATION
OBSERVATION_PERIOD
PAYER_PLAN_PERIOD
PERSON
PROCEDURE_OCCURRENCE
PROVIDER
RELATIONSHIP
SOURCE_TO_CONCEPT_MAP
SPECIMEN
VISIT_DETAIL
VISIT_OCCURRENCE
VOCABULARY


/var/folders/hq/659b4zd16vscxjxb3lxdr18c0000gp/T/ipykernel_64197/1350057838.py:3: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = Inspector.from_engine(engine)


In [100]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.schema import CreateTable
from sqlalchemy.engine.reflection import Inspector

# Assuming USERNAME, PASSWORD, SERVER, and DATABASE variables are defined above this line
connection_string = f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}'
engine = create_engine(connection_string, echo=True)

# Your existing code to execute SQL commands from a file...

# Now, let's print the DDL for each table
inspector = Inspector.from_engine(engine)

# Reflect the database schema into a new MetaData instance
metadata = MetaData()
metadata.reflect(bind=engine)

print("DDL for tables in the database:")
for table_name in inspector.get_table_names():
    # Using the reflected metadata to get the table object
    table = metadata.tables[table_name]
    # Generating the CREATE TABLE statement for the table
    ddl = CreateTable(table).compile(engine).string
    print(f"\nDDL for {table_name}:\n{ddl}")


2024-04-18 22:59:40,528 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-18 22:59:40,528 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:59:40,530 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-18 22:59:40,530 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:59:40,531 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-18 22:59:40,531 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:59:40,535 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 22:59:40,536 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `cdm`
2024-04-18 22:59:40,536 INFO sqlalchemy.engine.Engine [raw sql] {}


2024-04-18 22:59:40,546 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `CARE_SITE`
2024-04-18 22:59:40,547 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:59:40,552 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `CDM_SOURCE`
2024-04-18 22:59:40,553 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:59:40,557 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `COHORT`
2024-04-18 22:59:40,558 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:59:40,562 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `COHORT_DEFINITION`
2024-04-18 22:59:40,563 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:59:40,566 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `CONCEPT`
2024-04-18 22:59:40,567 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:59:40,568 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `CONCEPT_ANCESTOR`
2024-04-18 22:59:40,568 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:59:40,571 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `CONCEPT_CL

/var/folders/hq/659b4zd16vscxjxb3lxdr18c0000gp/T/ipykernel_64197/3633835634.py:12: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = Inspector.from_engine(engine)


In [101]:
import re

# Path to your SQL file containing CREATE INDEX statements
sql_file_path = './OMOPCDM_sql_server_5.4_indices.sql'

# Read SQL commands from the file and replace the placeholder
with open(sql_file_path, 'r') as file:
    # Read the entire file content
    sql_commands = file.read()
# Remove multiline comments
sql_commands = re.sub(r'/\*.*?\*/', '', sql_commands, flags=re.DOTALL)

# Replace the placeholder
sql_commands = sql_commands.replace('@cdmDatabaseSchema.', '')

# Split the modified SQL commands into individual commands
commands = sql_commands.split(';')

# Filter out empty commands and commands that are commented out
filtered_commands = []
for command in commands:
    # Trim whitespace and remove single-line comments
    stripped_command = command.strip()
    if stripped_command and not stripped_command.startswith('--'):
        filtered_commands.append(stripped_command)

# Execute each filtered command
with engine.begin() as connection:
    for command in filtered_commands:
        try:
            connection.execute(text(command))
        except Exception as e:
            print(f"An error occurred: {e}")
            print(f"While executing: {command}\n")

print("Index creation commands executed successfully.")

2024-04-18 22:59:46,109 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 22:59:46,114 INFO sqlalchemy.engine.Engine CREATE CLUSTERED INDEX idx_person_id ON person (person_id ASC)
2024-04-18 22:59:46,115 INFO sqlalchemy.engine.Engine [generated in 0.00250s] {}
An error occurred: (mysql.connector.errors.ProgrammingError) 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'CLUSTERED INDEX idx_person_id ON person (person_id ASC)' at line 1
[SQL: CREATE CLUSTERED INDEX idx_person_id ON person (person_id ASC)]
(Background on this error at: https://sqlalche.me/e/20/f405)
While executing: CREATE CLUSTERED INDEX idx_person_id ON person (person_id ASC)

2024-04-18 22:59:46,118 INFO sqlalchemy.engine.Engine CREATE INDEX idx_gender ON person (gender_concept_id ASC)
2024-04-18 22:59:46,118 INFO sqlalchemy.engine.Engine [generated in 0.00046s] {}
2024-04-18 22:59:46,151 INFO sqlalchemy.engine.En

In [66]:
# loading a single csv file to the table
'''
import pandas as pd
csv_file_path = 'cohort/condition_occurrence.csv'  # Update this to the path of your CSV file
data = pd.read_csv(csv_file_path)

table_name = 'condition_occurrence'
# schema = 'cdmDatabaseSchema'

# Load the DataFrame into MySQL
data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data has been successfully loaded into the database.")

table_name = 'condition_occurrence'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result'''

'\nimport pandas as pd\ncsv_file_path = \'cohort/condition_occurrence.csv\'  # Update this to the path of your CSV file\ndata = pd.read_csv(csv_file_path)\n\ntable_name = \'condition_occurrence\'\n# schema = \'cdmDatabaseSchema\'\n\n# Load the DataFrame into MySQL\ndata.to_sql(name=table_name, con=engine, if_exists=\'append\', index=False)\n\nprint("Data has been successfully loaded into the database.")\n\ntable_name = \'condition_occurrence\'\nquery = text(f"SELECT * FROM {table_name}")\nresult = pd.read_sql(query, con=engine)\nprint("Data retrieved from the database:")\nresult'

In [71]:
import pandas as pd
csv_file_path = 'cohort/observation.csv'  # Update this to the path of your CSV file
data = pd.read_csv(csv_file_path)

table_name = 'observation'
# schema = 'cdmDatabaseSchema'

# Load the DataFrame into MySQL
data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data has been successfully loaded into the database.")

query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 14:54:04,022 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 14:54:04,047 INFO sqlalchemy.engine.Engine DESCRIBE `cdm`.`observation`
2024-04-19 14:54:04,048 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-19 14:54:04,083 INFO sqlalchemy.engine.Engine INSERT INTO observation (observation_id, person_id, observation_concept_id, observation_date, observation_datetime, observation_type_concept_id, value_as_number, value_as_string, value_as_concept_id, qualifier_concept_id, unit_concept_id, provider_id, visit_occurrence_id, visit_detail_id, observation_source_value, observation_source_concept_id, unit_source_value, qualifier_source_value, value_source_value) VALUES (%(observation_id)s, %(person_id)s, %(observation_concept_id)s, %(observation_date)s, %(observation_datetime)s, %(observation_type_concept_id)s, %(value_as_number)s, %(value_as_string)s, %(value_as_concept_id)s, %(qualifier_concept_id)s, %(unit_concept_id)s, %(provider_id)s, %(visit_occurrence_id)s, %(visit

,observation_id,person_id,observation_concept_id,observation_date,observation_datetime,observation_type_concept_id,value_as_number,value_as_string,value_as_concept_id,qualifier_concept_id,...,provider_id,visit_occurrence_id,visit_detail_id,observation_source_value,observation_source_concept_id,unit_source_value,qualifier_source_value,value_source_value,observation_event_id,obs_event_field_concept_id
0,1002510856,1372782,294002,2020-03-13,2020-03-13 21:32:00,7460767,None,None,None,0,...,None,66217400,None,Z87.8,4403860,None,No matching concept,None,None,None
1,1002704901,1372782,2770454,2020-03-13,2020-03-13 21:32:00,7460767,None,None,None,0,...,None,66217400,None,Z677.22,5698920,None,No matching concept,None,None,None
2,1022704905,1372782,2759786,2020-03-13,2020-03-13 21:32:00,7460767,None,None,None,0,...,None,66217400,None,Z89,5694701,None,No matching concept,None,None,None
3,1042511804,1372782,31002266,2020-03-13,2020-03-13 21:32:00,7460767,None,None,None,0,...,None,66217400,None,Z45.51,5697712,None,No matching concept,None,None,None
4,1042693544,1372782,2841127,2020-03-13,2020-03-13 21:32:00,7460767,None,None,None,0,...,None,66217400,None,Z88.0,4403859,None,No matching concept,None,None,None
5,1062511800,1372782,31003260,2020-03-13,2020-03-13 21:32:00,7460767,None,None,None,0,...,None,66217400,None,Z79.01,5694703,None,No matching concept,None,None,None
6,1202664983,1372782,2693100,2020-03-13,2020-03-13 21:32:00,7460767,None,None,None,0,...,None,66217400,None,Z43.2,4403869,None,No matching concept,None,None,None
7,1562510832,1372782,2791757,2020-03-13,2020-03-13 21:32:00,7460767,None,None,None,0,...,None,66217400,None,Z80.7,4403853,None,No matching concept,None,None,None
8,1603093411,1372782,2823743,2020-03-13,2020-03-13 21:32:00,7460767,None,None,None,0,...,None,66217400,None,Z67.8,5701334,None,No matching concept,None,None,None
9,1672635332,1372782,2719982,2020-03-13,2020-03-13 21:32:00,7460767,None,None,None,0,...,None,66217400,None,Z95.2,4403866,None,No matching concept,None,None,None


In [72]:
import pandas as pd
csv_file_path = 'cohort/person.csv'
data = pd.read_csv(csv_file_path)

table_name = 'person'
# Load the DataFrame into MySQL
data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data has been successfully loaded into the database.")

query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 14:54:06,761 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 14:54:06,764 INFO sqlalchemy.engine.Engine DESCRIBE `cdm`.`person`
2024-04-19 14:54:06,766 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-19 14:54:06,770 INFO sqlalchemy.engine.Engine INSERT INTO person (person_id, gender_concept_id, year_of_birth, month_of_birth, day_of_birth, birth_datetime, race_concept_id, ethnicity_concept_id, location_id, provider_id, care_site_id, person_source_value, gender_source_value, gender_source_concept_id, race_source_value, race_source_concept_id, ethnicity_source_value, ethnicity_source_concept_id) VALUES (%(person_id)s, %(gender_concept_id)s, %(year_of_birth)s, %(month_of_birth)s, %(day_of_birth)s, %(birth_datetime)s, %(race_concept_id)s, %(ethnicity_concept_id)s, %(location_id)s, %(provider_id)s, %(care_site_id)s, %(person_source_value)s, %(gender_source_value)s, %(gender_source_concept_id)s, %(race_source_value)s, %(race_source_concept_id)s, %(ethnicity_source_valu

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,680331,41310599,1946,2,27,1946-02-27,66754,37926811,None,None,None,259008,GenderIdentity_Woman,907276,WhatRaceEthnicity_White,4567549,Not Hispanic,31460238
1,711599,1995432138,1978,4,6,1978-04-06,817420,826343,None,None,None,305678,GenderIdentity_GeneralizedDiffGender,1233546570,PMI_Skip,67856,PMI_Skip,67856
2,872883,41312805,1989,11,20,1989-11-20,66754,37926811,None,None,None,546399,GenderIdentity_Man,907275,WhatRaceEthnicity_Black,4567549,Not Hispanic,31460238
3,1021059,41310599,1969,1,20,1969-01-20,66754,37926811,None,None,None,767558,GenderIdentity_Woman,907276,WhatRaceEthnicity_Black,4567549,Not Hispanic,31460238
4,1372782,1995432138,1976,6,1,1976-06-01,1099923437,37926810,None,None,None,367286,GenderIdentity_GeneralizedDiffGender,1233546570,AoUDRC_NoneIndicated,0,Hispanic,31460237
5,1543037,1995432138,1968,9,3,1968-09-03,817420,826343,None,None,None,1546630,GenderIdentity_GeneralizedDiffGender,1233546570,PMI_Skip,67856,PMI_Skip,67856
6,1851447,41310599,1964,12,10,1964-12-10,1099923437,37926810,None,None,None,2006943,GenderIdentity_Woman,907276,AoUDRC_NoneIndicated,0,Hispanic,31460237
7,1955206,41312805,1959,8,5,1959-08-05,1099923437,37926810,None,None,None,2161807,GenderIdentity_Man,907275,AoUDRC_NoneIndicated,0,Hispanic,31460237
8,2146445,41312805,1951,5,15,1951-05-15,66754,37926811,None,None,None,2447238,GenderIdentity_Man,907275,WhatRaceEthnicity_Black,4567549,Not Hispanic,31460238
9,2334201,1995432138,1965,7,16,1965-07-16,817420,826343,None,None,None,2727471,GenderIdentity_GeneralizedDiffGender,1233546570,PMI_Skip,67856,PMI_Skip,67856


In [73]:
import pandas as pd
csv_file_path = 'cohort/condition_occurrence.csv'  # Update this to the path of your CSV file
data = pd.read_csv(csv_file_path)

table_name = 'condition_occurrence'
# schema = 'cdmDatabaseSchema'

# Load the DataFrame into MySQL
data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data has been successfully loaded into the database.")

query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 14:54:09,798 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 14:54:09,803 INFO sqlalchemy.engine.Engine DESCRIBE `cdm`.`condition_occurrence`
2024-04-19 14:54:09,804 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-19 14:54:09,812 INFO sqlalchemy.engine.Engine INSERT INTO condition_occurrence (condition_occurrence_id, person_id, condition_concept_id, condition_start_date, condition_start_datetime, condition_end_date, condition_end_datetime, condition_type_concept_id, condition_status_concept_id, stop_reason, provider_id, visit_occurrence_id, visit_detail_id, condition_source_value, condition_source_concept_id, condition_status_source_value) VALUES (%(condition_occurrence_id)s, %(person_id)s, %(condition_concept_id)s, %(condition_start_date)s, %(condition_start_datetime)s, %(condition_end_date)s, %(condition_end_datetime)s, %(condition_type_concept_id)s, %(condition_status_concept_id)s, %(stop_reason)s, %(provider_id)s, %(visit_occurrence_id)s, %(visit_detail_id)s

,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,condition_status_concept_id,stop_reason,provider_id,visit_occurrence_id,visit_detail_id,condition_source_value,condition_source_concept_id,condition_status_source_value
0,66677,5675534,319835,2018-08-04,2018-08-04 10:43:00,None,None,24498375,3680479,None,None,66217400,None,I50.43,35201517,68094332
1,238140,1372782,4043371,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,K52,35201517,68094332
2,248088,1372782,196152,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,K65.8,35201569,68094332
3,276646,1372782,37110250,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,I70.3,45541306,68094332
4,276647,1372782,317577,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,I71.2,45560456,68094332
5,289289,1372782,197596,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,K52,35201517,68094332
6,289385,1372782,319835,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,I50.21,45536428,68094332
7,289387,1372782,193782,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,N19,45541899,68094332
8,301436,1372782,4000609,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,K40,35201492,68094332
9,306327,1372782,4281826,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,D72.2,45598626,68094332


In [75]:
import pandas as pd
csv_file_path = 'cohort/visit_occurrence.csv'  # Update this to the path of your CSV file
data = pd.read_csv(csv_file_path)

table_name = 'visit_occurrence'
# schema = 'cdmDatabaseSchema'

# Load the DataFrame into MySQL
data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data has been successfully loaded into the database.")

query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 14:54:30,675 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 14:54:30,690 INFO sqlalchemy.engine.Engine DESCRIBE `cdm`.`visit_occurrence`
2024-04-19 14:54:30,691 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-19 14:54:30,724 INFO sqlalchemy.engine.Engine INSERT INTO visit_occurrence (visit_occurrence_id, person_id, visit_concept_id, visit_start_date, visit_start_datetime, visit_end_date, visit_end_datetime, visit_type_concept_id, provider_id, care_site_id, visit_source_value, visit_source_concept_id, admitted_from_concept_id, admitted_from_source_value, discharged_to_concept_id, discharged_to_source_value, preceding_visit_occurrence_id) VALUES (%(visit_occurrence_id)s, %(person_id)s, %(visit_concept_id)s, %(visit_start_date)s, %(visit_start_datetime)s, %(visit_end_date)s, %(visit_end_datetime)s, %(visit_type_concept_id)s, %(provider_id)s, %(care_site_id)s, %(visit_source_value)s, %(visit_source_concept_id)s, %(admitted_from_concept_id)s, %(admitted_from_source_

,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitted_from_concept_id,admitted_from_source_value,discharged_to_concept_id,discharged_to_source_value,preceding_visit_occurrence_id
0,8790700,3651691,9202,2016-07-20,2016-07-19 16:48:00,2017-09-19,2017-09-18 16:48:00,39371665,None,None,15,1933,6973,5,6513,7,None
1,9552000,2333210,9202,2015-11-06,2015-11-05 16:48:00,2017-03-06,2017-03-05 16:48:00,39371665,None,None,15,1933,6973,5,6513,7,None
2,39627400,2680855,9202,2014-11-26,2014-11-25 16:48:00,2016-05-26,2016-05-25 16:48:00,39371665,None,None,15,1933,6973,5,6513,7,None
3,40848600,3924012,9202,2015-08-08,2015-08-07 16:48:00,2016-09-08,2016-09-07 16:48:00,39371665,None,None,15,1933,6973,5,6513,7,None
4,42780100,1503632,9202,2015-08-13,2015-08-12 16:48:00,2016-10-13,2016-10-12 16:48:00,39371665,None,None,15,1933,6973,5,6513,7,None
5,43037600,3590868,9202,2016-03-01,2016-02-29 16:48:00,2017-04-01,2017-03-31 16:48:00,39371665,None,None,15,1933,6973,5,6513,7,None
6,43140500,3485274,9202,2016-03-22,2016-03-21 19:12:00,2017-04-21,2017-04-20 16:48:00,39371665,None,None,15,1933,6973,5,6513,7,None
7,43246200,1667878,9202,2016-03-29,2016-03-28 19:12:00,2017-06-29,2017-06-28 16:48:00,39371665,None,None,15,1933,6973,5,6513,7,None
8,43552600,3743446,9202,2016-03-16,2016-03-15 16:48:00,2016-11-22,2016-11-21 16:48:00,39371665,None,None,15,1933,6973,5,6513,7,None
9,66217400,2746928,262,2018-06-27,2018-06-26 16:48:00,2019-08-17,2019-08-16 16:48:00,39371665,None,None,15,1933,6973,5,44812663,Discharge Disposition-E,None


In [76]:
import pandas as pd
csv_file_path = 'cohort/drug_exposure.csv'  # Update this to the path of your CSV file
data = pd.read_csv(csv_file_path)

table_name = 'drug_exposure'
# schema = 'cdmDatabaseSchema'

# Load the DataFrame into MySQL
data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data has been successfully loaded into the database.")

query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 14:54:33,499 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 14:54:33,501 INFO sqlalchemy.engine.Engine DESCRIBE `cdm`.`drug_exposure`
2024-04-19 14:54:33,502 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-19 14:54:33,507 INFO sqlalchemy.engine.Engine INSERT INTO drug_exposure (drug_exposure_id, person_id, drug_concept_id, drug_exposure_start_date, drug_exposure_start_datetime, drug_exposure_end_date, drug_exposure_end_datetime, verbatim_end_date, drug_type_concept_id, stop_reason, refills, quantity, days_supply, sig, route_concept_id, lot_number, provider_id, visit_occurrence_id, visit_detail_id, drug_source_value, drug_source_concept_id, route_source_value, dose_unit_source_value) VALUES (%(drug_exposure_id)s, %(person_id)s, %(drug_concept_id)s, %(drug_exposure_start_date)s, %(drug_exposure_start_datetime)s, %(drug_exposure_end_date)s, %(drug_exposure_end_datetime)s, %(verbatim_end_date)s, %(drug_type_concept_id)s, %(stop_reason)s, %(refills)s, %(quantity)s, 

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,...,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value
0,3456,5675534,36249642,2018-08-04,2018-08-04 09:05:00,2018-08-04,2018-08-04 09:05:00,None,16943435,None,...,None,None,None,None,66252165,None,307247,879455,11722356,None
1,86043,5675534,36249642,2018-08-22,2018-08-22 01:46:00,2018-08-22,2018-08-22 01:46:00,None,16943435,None,...,None,None,None,None,66252165,None,307247,879455,11722356,None
2,94533,5675534,36249642,2018-08-04,2018-08-04 10:08:00,2018-08-04,2018-08-04 10:08:00,None,16943435,None,...,None,None,None,None,66252165,None,307247,879455,11722356,None
3,156714,1372782,19072176,2020-01-23,2020-01-23 00:00:00,2021-06-17,2021-06-17 09:36:00,None,16943435,None,...,None,None,None,None,66252165,None,301140,14876354,11722356,None
4,156731,1372782,19079524,2020-01-24,2020-01-24 00:00:00,2020-04-11,2020-04-11 09:36:00,None,16943435,None,...,None,None,None,None,66252165,None,306467,14882086,20954437,None
5,156742,1372782,19049106,2020-04-26,2020-04-26 00:00:00,2020-03-23,2020-06-22 19:12:00,None,16943435,None,...,None,None,None,None,66252165,None,191581,14858360,11722356,None
6,156757,1372782,19127952,2020-05-27,2020-05-27 00:00:00,2020-03-23,2020-07-22 09:36:00,None,16943435,None,...,None,None,None,None,66252165,None,742671,14919860,11722356,None
7,156766,1372782,19130823,2020-01-29,2020-01-29 00:00:00,2020-05-07,2020-05-07 09:36:00,None,16943435,None,...,None,None,None,None,66252165,None,793937,14922099,16824594,None
8,156786,1372782,1127433,2020-01-30,2020-01-30 00:00:00,2020-03-26,2020-03-26 09:36:00,None,16943435,None,...,None,None,None,None,66252165,None,307247,879455,11722356,None
9,156828,1372782,40167259,2020-02-02,2020-02-02 00:00:00,2021-12-10,2021-12-10 09:36:00,None,16943435,None,...,None,None,None,None,66252165,None,869658,31330519,11722356,None


In [77]:
import pandas as pd
csv_file_path = 'cohort/device_exposure.csv'  # Update this to the path of your CSV file
data = pd.read_csv(csv_file_path)

table_name = 'device_exposure'
# schema = 'cdmDatabaseSchema'

# Load the DataFrame into MySQL
data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data has been successfully loaded into the database.")

query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 14:54:36,471 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 14:54:36,474 INFO sqlalchemy.engine.Engine DESCRIBE `cdm`.`device_exposure`
2024-04-19 14:54:36,476 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-19 14:54:36,481 INFO sqlalchemy.engine.Engine INSERT INTO device_exposure (device_exposure_id, person_id, device_concept_id, device_exposure_start_date, device_exposure_start_datetime, device_exposure_end_date, device_exposure_end_datetime, device_type_concept_id, unique_device_id, quantity, provider_id, visit_occurrence_id, visit_detail_id, device_source_value, device_source_concept_id) VALUES (%(device_exposure_id)s, %(person_id)s, %(device_concept_id)s, %(device_exposure_start_date)s, %(device_exposure_start_datetime)s, %(device_exposure_end_date)s, %(device_exposure_end_datetime)s, %(device_type_concept_id)s, %(unique_device_id)s, %(quantity)s, %(provider_id)s, %(visit_occurrence_id)s, %(visit_detail_id)s, %(device_source_value)s, %(device_source_concep

,device_exposure_id,person_id,device_concept_id,device_exposure_start_date,device_exposure_start_datetime,device_exposure_end_date,device_exposure_end_datetime,device_type_concept_id,unique_device_id,production_id,quantity,provider_id,visit_occurrence_id,visit_detail_id,device_source_value,device_source_concept_id,unit_concept_id,unit_source_value,unit_source_concept_id
0,417,493797,4097216,2021-01-08,2021-01-08 14:18:00,2021-01-08,2021-01-07 20:04:00,6785,None,None,NaN,None,56788.0,None,None,NaN,None,None,None
1,5675,2915590,44844102,2021-11-25,2021-11-24 23:00:00,None,NaT,0,None,None,100.0,None,NaN,None,11841481096.0,38099367.0,None,None,None
2,12375,2915590,44908056,2017-04-24,2017-04-24 00:34:20,2017-07-22,2017-07-22 00:34:45,0,None,None,100.0,None,32612.0,None,118198900.0,38163321.0,None,None,None
3,28284,493797,4224038,2021-09-09,2021-09-09 12:25:00,None,NaT,6785,None,None,NaN,None,NaN,None,261090688.0,3549565.0,None,None,None
4,432678,493797,4224038,2021-09-09,2021-09-09 13:20:00,2023-07-02,2023-07-02 03:45:00,6785,None,None,NaN,None,654040.0,None,261090688.0,3549561.0,None,None,None
5,467858,2915590,44908056,2017-04-24,2017-04-24 14:15:24,2017-07-22,2017-07-22 14:15:34,0,None,None,100.0,None,5785612.0,None,118198900.0,38163321.0,None,None,None
6,843098,2915590,44844102,2020-09-14,2020-09-13 23:00:00,2022-01-11,2022-01-11 15:16:00,0,None,None,NaN,None,NaN,None,11841481096.0,38099367.0,None,None,None
7,973871,493797,4224038,2021-01-09,2021-01-09 03:00:00,2021-01-14,2021-01-14 17:39:00,6785,None,None,NaN,None,3300546.0,None,261090688.0,3549561.0,None,None,None


In [103]:
import pandas as pd
csv_file_path = 'cohort/concept.csv'  # Update this to the path of your CSV file
data = pd.read_csv(csv_file_path)

table_name = 'concept'
# schema = 'cdmDatabaseSchema'

# Load the DataFrame into MySQL
data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data has been successfully loaded into the database.")

query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 15:48:08,729 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 15:48:08,751 INFO sqlalchemy.engine.Engine DESCRIBE `cdm`.`concept`
2024-04-19 15:48:08,752 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-19 15:48:08,814 INFO sqlalchemy.engine.Engine INSERT INTO concept (concept_id, concept_name, domain_id, vocabulary_id, concept_class_id, standard_concept, concept_code, valid_start_date, valid_end_date, invalid_reason) VALUES (%(concept_id)s, %(concept_name)s, %(domain_id)s, %(vocabulary_id)s, %(concept_class_id)s, %(standard_concept)s, %(concept_code)s, %(valid_start_date)s, %(valid_end_date)s, %(invalid_reason)s)
2024-04-19 15:48:08,815 INFO sqlalchemy.engine.Engine [generated in 0.02736s] [{'concept_id': 8524, 'concept_name': 'Outpatient Visit', 'domain_id': 'Visit', 'vocabulary_id': 'Visit', 'concept_class_id': 'Visit', 'standard_concept': 'S', 'concept_code': 'OP', 'valid_start_date': '2015-07-26', 'valid_end_date': '2048-02-02', 'invalid_reason': None}, {'con

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,262,Emergency Room and Inpatient Visit,Visit,Visit,Visit,S,ERIP,1970-01-01,2099-12-31,None
1,8516,White,race,R,x,s,1801294,2016-08-02,2100-01-01,None
2,8524,Outpatient Visit,Visit,Visit,Visit,S,OP,2015-07-26,2048-02-02,None
3,8717,Inpatient Hospital,Visit,CMS Place of Service,Visit,S,21,1970-01-01,2099-12-31,None
4,9202,Outpatient Visit,Visit,Visit,Visit,S,OP,1970-01-01,2099-12-31,None
5,32817,EHR,Type Concept,Type Concept,Type Concept,S,OMOP4976890,2020-08-20,2099-12-31,None
6,66754,Black Or African American,race,s,s,s,1801294,2016-08-03,2100-01-02,None
7,193782,End-stage renal disease,Condition,LOINC,Lab Test,S,39791-17,2008-08-10,2039-12-23,None
8,196152,Peritonitis,Condition,LOINC,Lab Test,S,39791-10,2008-08-03,2039-12-16,None
9,197596,Toxic gastroenteritis,Condition,LOINC,Lab Test,S,39791-16,2008-08-09,2039-12-22,None


In [83]:
import pandas as pd
csv_file_path = 'cohort/measurement.csv'  # Update this to the path of your CSV file
data = pd.read_csv(csv_file_path)

table_name = 'measurement'
# schema = 'cdmDatabaseSchema'

# Load the DataFrame into MySQL
data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data has been successfully loaded into the database.")

query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 15:03:49,123 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 15:03:49,161 INFO sqlalchemy.engine.Engine DESCRIBE `cdm`.`measurement`
2024-04-19 15:03:49,167 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-19 15:03:49,227 INFO sqlalchemy.engine.Engine INSERT INTO measurement (measurement_id, person_id, measurement_concept_id, measurement_date, measurement_datetime, measurement_time, measurement_type_concept_id, operator_concept_id, value_as_number, value_as_concept_id, unit_concept_id, range_low, range_high, provider_id, visit_occurrence_id, visit_detail_id, measurement_source_value, measurement_source_concept_id, unit_source_value, value_source_value) VALUES (%(measurement_id)s, %(person_id)s, %(measurement_concept_id)s, %(measurement_date)s, %(measurement_datetime)s, %(measurement_time)s, %(measurement_type_concept_id)s, %(operator_concept_id)s, %(value_as_number)s, %(value_as_concept_id)s, %(unit_concept_id)s, %(range_low)s, %(range_high)s, %(provider_id)s, %(

,measurement_id,person_id,measurement_concept_id,measurement_date,measurement_datetime,measurement_time,measurement_type_concept_id,operator_concept_id,value_as_number,value_as_concept_id,...,provider_id,visit_occurrence_id,visit_detail_id,measurement_source_value,measurement_source_concept_id,unit_source_value,unit_source_concept_id,value_source_value,measurement_event_id,meas_event_field_concept_id
0,58776,5675534,3023103,2017-01-01,2016-12-31 20:40:00,None,44818702,4172703.0,4.0,None,...,None,12504473,None,2823-3,3023103,mmol/L,None,None,None,None
1,74577,5675534,3023103,2018-08-07,2018-08-07 02:38:00,None,44818702,4172703.0,4.2,None,...,None,68704051,None,2823-3,3023103,mmol/L,None,None,None,None
2,435677,5675534,3023103,2018-08-22,2018-08-22 02:27:00,None,44818702,4172703.0,4.4,None,...,None,22357735,None,2823-3,3023103,mmol/L,None,None,None,None
3,459800,2217111,3027172,2022-04-07,2022-04-07 13:32:00,None,3026494,NaN,14.0,None,...,None,144636930,None,2023-1,3026494,%,None,None,None,None
4,612700,11549211,3027172,2020-12-15,2020-12-15 13:55:00,None,44818024,0.0,56.0,None,...,None,284959,None,2023-1,3026494,%,None,None,None,None
5,3475780,5675534,3023103,2018-08-20,2018-08-20 03:36:00,None,44818702,4172703.0,3.8,None,...,None,56640292,None,2823-3,3023103,mmol/L,None,None,None,None
6,5688646,5675534,3023103,2018-08-04,2018-08-04 07:06:00,None,32489,4172703.0,3.1,None,...,None,704051,None,2823-3,3023103,mmol/L,None,None,None,None
7,6151470,1749910,3027172,2023-09-17,2023-09-17 13:22:00,None,3026494,NaN,0.0,None,...,None,146059710,None,2023-1,3026494,%,None,None,None,None
8,6435436,5675534,3023103,2018-08-21,2018-08-21 05:59:00,None,32489,4172703.0,3.8,None,...,None,15517735,None,2823-3,3023103,mmol/L,None,None,None,None
9,7298130,1874170,3027172,2019-12-21,2019-12-22 14:09:05,None,3026494,NaN,63.2,None,...,None,5263240,None,2023-1,3026494,%,None,None,None,None


In [106]:
import pandas as pd
csv_file_path = 'cohort/procedure_occurrence.csv'  # Update this to the path of your CSV file
data = pd.read_csv(csv_file_path)

table_name = 'procedure_occurrence'
# schema = 'cdmDatabaseSchema'

# Load the DataFrame into MySQL
data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data has been successfully loaded into the database.")

query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 15:55:48,152 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 15:55:48,171 INFO sqlalchemy.engine.Engine DESCRIBE `cdm`.`procedure_occurrence`
2024-04-19 15:55:48,172 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-19 15:55:48,223 INFO sqlalchemy.engine.Engine INSERT INTO procedure_occurrence (procedure_occurrence_id, person_id, procedure_concept_id, procedure_date, procedure_datetime, procedure_type_concept_id, modifier_concept_id, quantity, provider_id, visit_occurrence_id, visit_detail_id, procedure_source_value, procedure_source_concept_id, modifier_source_value) VALUES (%(procedure_occurrence_id)s, %(person_id)s, %(procedure_concept_id)s, %(procedure_date)s, %(procedure_datetime)s, %(procedure_type_concept_id)s, %(modifier_concept_id)s, %(quantity)s, %(provider_id)s, %(visit_occurrence_id)s, %(visit_detail_id)s, %(procedure_source_value)s, %(procedure_source_concept_id)s, %(modifier_source_value)s)
2024-04-19 15:55:48,225 INFO sqlalchemy.engine.Engine [gener

,procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_datetime,procedure_end_date,procedure_end_datetime,procedure_type_concept_id,modifier_concept_id,quantity,provider_id,visit_occurrence_id,visit_detail_id,procedure_source_value,procedure_source_concept_id,modifier_source_value
0,11289,1861428,2746520,2020-02-27,2020-02-27,None,None,36429305,0,None,None,8790700,None,0DB18ZX,2746520,No matching concept
1,11443,1861428,2746773,2020-02-27,2020-02-27,None,None,36429305,0,None,None,66217400,None,0DB68ZX,2746773,No matching concept
2,26955,1861428,2747274,2020-02-27,2020-02-27,None,None,36429305,0,None,None,66217400,None,0DBK8ZX,2747274,No matching concept
3,30589,1861428,2732505,2020-03-09,2020-03-09,None,None,36429305,0,None,None,66217400,None,047Q3ZZ,2732505,No matching concept
4,34474,1861428,2746803,2020-02-27,2020-02-27,None,None,36429305,0,None,None,66217400,None,0DB98ZX,2746803,No matching concept
5,38182,1861428,2788126,2020-03-09,2020-03-09,None,None,36429305,0,None,None,8790700,None,B41FYZZ,2788126,No matching concept
6,43885,1861428,2788717,2020-03-11,2020-03-11,None,None,36429305,0,None,None,66217400,None,30233N1,2788717,No matching concept
7,58054,1861428,2749338,2020-03-14,2020-03-14,None,None,36429305,0,None,None,8790700,None,0DJW4ZZ,2749338,No matching concept
8,84354,1861428,2747017,2020-02-27,2020-02-27,None,None,36429305,0,None,None,66217400,None,0DBB8ZX,2747017,No matching concept


In [62]:
## Insert all the data

import pandas as pd
from sqlalchemy import create_engine, exc
import os
directory_path = './cohort/'
csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
print(csv_files)
# Loop through the CSV files and load each one into the corresponding MySQL table
for file_name in csv_files:
    table_name = file_name[:-4]  # Remove the '.csv' part to get the table name
    csv_file_path = os.path.join(directory_path, file_name)
    
    try:
        print(f'Loading data from {table_name}')
        # Read the CSV file into a DataFrame
        data = pd.read_csv(csv_file_path)

        # Convert NaT in datetime columns to None (if applicable)
        datetime_columns = data.select_dtypes(include=['datetime']).columns
        data[datetime_columns] = data[datetime_columns].where(data[datetime_columns].notna(), None)

        # Load the DataFrame into MySQL
        data.to_sql(name=table_name, con=engine, if_exists='append', index=False)
        print(f"Data from {file_name} has been successfully loaded into the {table_name} table in the database.")
    
    except exc.IntegrityError as e:
        print(f"IntegrityError: {e}. Skipping duplicate entries for {table_name}.")
    except Exception as e:
        print(f"An error occurred with {table_name}: {e}")

print("All data has been loaded into the database.")


['drug_exposure.csv', 'device_exposure.csv', 'concept.csv', 'measurement.csv', 'condition_occurrence.csv', 'visit_occurrence.csv', 'person.csv', 'observation.csv', 'procedure_occurrence.csv']
Loading data from drug_exposure
2024-04-19 14:51:13,171 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 14:51:13,215 INFO sqlalchemy.engine.Engine DESCRIBE `cdm`.`drug_exposure`
2024-04-19 14:51:13,217 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-19 14:51:13,305 INFO sqlalchemy.engine.Engine INSERT INTO drug_exposure (drug_exposure_id, person_id, drug_concept_id, drug_exposure_start_date, drug_exposure_start_datetime, drug_exposure_end_date, drug_exposure_end_datetime, verbatim_end_date, drug_type_concept_id, stop_reason, refills, quantity, days_supply, sig, route_concept_id, lot_number, provider_id, visit_occurrence_id, visit_detail_id, drug_source_value, drug_source_concept_id, route_source_value, dose_unit_source_value) VALUES (%(drug_exposure_id)s, %(person_id)s, %(drug_concept

In [ ]:
import pandas
import os
sql = """SELECT c.person_id,
       c.condition_occurrence_id,
       c.condition_concept_id,
       c.condition_start_date,
       c.condition_end_date,
       d.drug_exposure_id,
       d.drug_concept_id,
       d.drug_exposure_start_date,
       d.drug_exposure_end_date
FROM CONDITION_OCCURRENCE c
INNER JOIN PERSON p ON c.person_id = p.person_id
INNER JOIN DRUG_EXPOSURE d ON p.person_id = d.person_id"""#LVEF
df = pandas.read_gbq(
    sql,
    dialect="standard",
    # use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")
df.head(10)

In [87]:
table_name = 'PERSON'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 15:05:30,083 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 15:05:30,084 INFO sqlalchemy.engine.Engine SELECT * FROM PERSON
2024-04-19 15:05:30,085 INFO sqlalchemy.engine.Engine [generated in 0.00323s] {}
2024-04-19 15:05:30,110 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,680331,41310599,1946,2,27,1946-02-27,66754,37926811,None,None,None,259008,GenderIdentity_Woman,907276,WhatRaceEthnicity_White,4567549,Not Hispanic,31460238
1,711599,1995432138,1978,4,6,1978-04-06,817420,826343,None,None,None,305678,GenderIdentity_GeneralizedDiffGender,1233546570,PMI_Skip,67856,PMI_Skip,67856
2,872883,41312805,1989,11,20,1989-11-20,66754,37926811,None,None,None,546399,GenderIdentity_Man,907275,WhatRaceEthnicity_Black,4567549,Not Hispanic,31460238
3,1021059,41310599,1969,1,20,1969-01-20,66754,37926811,None,None,None,767558,GenderIdentity_Woman,907276,WhatRaceEthnicity_Black,4567549,Not Hispanic,31460238
4,1372782,1995432138,1976,6,1,1976-06-01,1099923437,37926810,None,None,None,367286,GenderIdentity_GeneralizedDiffGender,1233546570,AoUDRC_NoneIndicated,0,Hispanic,31460237
5,1543037,1995432138,1968,9,3,1968-09-03,817420,826343,None,None,None,1546630,GenderIdentity_GeneralizedDiffGender,1233546570,PMI_Skip,67856,PMI_Skip,67856
6,1851447,41310599,1964,12,10,1964-12-10,1099923437,37926810,None,None,None,2006943,GenderIdentity_Woman,907276,AoUDRC_NoneIndicated,0,Hispanic,31460237
7,1955206,41312805,1959,8,5,1959-08-05,1099923437,37926810,None,None,None,2161807,GenderIdentity_Man,907275,AoUDRC_NoneIndicated,0,Hispanic,31460237
8,2146445,41312805,1951,5,15,1951-05-15,66754,37926811,None,None,None,2447238,GenderIdentity_Man,907275,WhatRaceEthnicity_Black,4567549,Not Hispanic,31460238
9,2334201,1995432138,1965,7,16,1965-07-16,817420,826343,None,None,None,2727471,GenderIdentity_GeneralizedDiffGender,1233546570,PMI_Skip,67856,PMI_Skip,67856


In [89]:
import pandas
query = text("""SELECT p.person_id,
       p.gender_concept_id,
       p.year_of_birth,
       p.month_of_birth,
       p.day_of_birth,
       p.race_concept_id,
       p.ethnicity_concept_id,
       d.drug_exposure_id,
       d.drug_concept_id,
       d.drug_exposure_start_date,
       d.drug_exposure_end_date,
       c.condition_occurrence_id,
       c.condition_concept_id,
       c.condition_start_date,
       c.condition_end_date,
       cc.concept_name AS condition_name,
       dc.concept_name AS drug_name
FROM DRUG_EXPOSURE d
INNER JOIN PERSON p ON d.person_id = p.person_id
LEFT JOIN CONDITION_OCCURRENCE c ON d.person_id = c.person_id
LEFT JOIN CONCEPT cc ON c.condition_concept_id = cc.concept_id
LEFT JOIN CONCEPT dc ON d.drug_concept_id = dc.concept_id;""")

result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
type(result)

2024-04-19 15:05:47,434 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 15:05:47,434 INFO sqlalchemy.engine.Engine SELECT p.person_id,
       p.gender_concept_id,
       p.year_of_birth,
       p.month_of_birth,
       p.day_of_birth,
       p.race_concept_id,
       p.ethnicity_concept_id,
       d.drug_exposure_id,
       d.drug_concept_id,
       d.drug_exposure_start_date,
       d.drug_exposure_end_date,
       c.condition_occurrence_id,
       c.condition_concept_id,
       c.condition_start_date,
       c.condition_end_date,
       cc.concept_name AS condition_name,
       dc.concept_name AS drug_name
FROM DRUG_EXPOSURE d
INNER JOIN PERSON p ON d.person_id = p.person_id
LEFT JOIN CONDITION_OCCURRENCE c ON d.person_id = c.person_id
LEFT JOIN CONCEPT cc ON c.condition_concept_id = cc.concept_id
LEFT JOIN CONCEPT dc ON d.drug_concept_id = dc.concept_id;
2024-04-19 15:05:47,435 INFO sqlalchemy.engine.Engine [cached since 1.709e+04s ago] {}
2024-04-19 15:05:47,443 INFO sqla

pandas.core.frame.DataFrame

In [90]:
result.to_csv('person_drug.csv')

In [ ]:
create_table_command = """
                    CREATE TABLE con_drug AS
                    SELECT p.person_id,
                        p.birth_datetime,
                        c2.concept_name AS gender,
                        c1.concept_name AS race,
                        cc.concept_name AS condition_name,
                        c.condition_start_date,
                        c.condition_end_date,
                        dc.concept_name AS drug_name,
                        d.drug_exposure_start_date,
                        d.drug_exposure_end_date
                    FROM DRUG_EXPOSURE d
                    INNER JOIN PERSON p ON d.person_id = p.person_id
                    LEFT JOIN CONDITION_OCCURRENCE c ON d.person_id = c.person_id
                    LEFT JOIN CONCEPT c1 ON p.race_concept_id = c1.concept_id
                    LEFT JOIN CONCEPT c2 ON p.gender_concept_id = c2.concept_id
                    LEFT JOIN CONCEPT cc ON c.condition_concept_id = cc.concept_id
                    LEFT JOIN CONCEPT dc ON d.drug_concept_id = dc.concept_id;
                    """
# Execute the create table command
with engine.begin() as conn:
    conn.execute(text(create_table_command))

In [ ]:
create_table_command = """CREATE TABLE combined_person_measurement AS
SELECT p.person_id, c1.concept_name AS gender, p.birth_datetime, c2.concept_name AS race, m.measurement_id,
    m.person_id AS measurement_person_id, mc.concept_name AS measurement_concept_name, m.measurement_datetime,
    m.value_as_number, m.range_low, m.range_high, m.unit_source_value
FROM
    PERSON p
JOIN
    MEASUREMENT m ON p.person_id = m.person_id
LEFT JOIN
    CONCEPT c1 ON p.gender_concept_id = c1.concept_id
LEFT JOIN
    CONCEPT c2 ON p.race_concept_id = c2.concept_id
LEFT JOIN
    CONCEPT mc ON m.measurement_concept_id = mc.concept_id;"""
with engine.begin() as conn:
    conn.execute(text(create_table_command))

In [ ]:
create_table_command = """CREATE TABLE combined_procedure_visit AS
SELECT vo.visit_occurrence_id, c1.concept_name AS visit_concept_name, vo.visit_start_date, 
    vo.visit_end_date, vo.care_site_id, p.person_id, p.procedure_occurrence_id,
    c2.concept_name AS pocedure_name, p.procedure_date, p.procedure_source_value
FROM
    VISIT_OCCURRENCE vo
JOIN
    PROCEDURE_OCCURRENCE p ON vo.visit_occurrence_id = p.visit_occurrence_id
JOIN
    CONCEPT c1 ON vo.visit_concept_id = c1.concept_id
JOIN
    CONCEPT c2 ON p.procedure_concept_id = c2.concept_id;
"""
with engine.begin() as conn:
    conn.execute(text(create_table_command))

In [91]:
table_name = 'con_drug'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 15:05:57,345 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 15:05:57,347 INFO sqlalchemy.engine.Engine SELECT * FROM con_drug
2024-04-19 15:05:57,348 INFO sqlalchemy.engine.Engine [cached since 1.377e+04s ago] {}
2024-04-19 15:05:57,380 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,person_id,gender_concept_id,birth_datetime,gender,drug_exposure_start_date,drug_concept_name,condition_start_date,condition_concept_name
0,1372782,1995432138,1976-06-01,Female,2020-02-13,100 ML metronidazole 5 MG/ML Injection,2020-03-13,Peritonitis
1,1372782,1995432138,1976-06-01,Female,2020-02-11,budesonide 3 MG Delayed Release Oral Capsule,2020-03-13,Peritonitis
2,1372782,1995432138,1976-06-01,Female,2020-02-06,2.5 ML argatroban 100 MG/ML Injection,2020-03-13,Peritonitis
3,1372782,1995432138,1976-06-01,Female,2020-02-02,polyethylene glycol 3350 17000 MG Powder for O...,2020-03-13,Peritonitis
4,1372782,1995432138,1976-06-01,Female,2020-01-30,acetaminophen 325 MG Oral Tablet,2020-03-13,Peritonitis
...,...,...,...,...,...,...,...,...
67,1372782,1995432138,1976-06-01,Female,2020-01-30,acetaminophen 325 MG Oral Tablet,2020-03-13,Hypervolemia
68,1372782,1995432138,1976-06-01,Female,2020-01-27,sevelamer carbonate 800 MG Oral Tablet,2020-03-13,Hypervolemia
69,1372782,1995432138,1976-06-01,Female,2020-01-26,potassium chloride 20 MEQ Extended Release Ora...,2020-03-13,Hypervolemia
70,1372782,1995432138,1976-06-01,Female,2020-01-24,odium chloride 9 MG/ML Injectable Solution,2020-03-13,Hypervolemia


And further if we combine with concept id we can know the actual name of drug or other things:
CREATE TABLE con_drug AS    
SELECT p.person_id,
    p.birth_datetime,
    c2.concept_name as gender,
    c1.concept_name as race,
    cc.concept_name AS condition_name,
    c.condition_start_date,
    c.condition_end_date,
    dc.concept_name AS drug_name,
    d.drug_exposure_start_date,
    d.drug_exposure_end_date
FROM DRUG_EXPOSURE d
INNER JOIN PERSON p ON d.person_id = p.person_id
LEFT JOIN CONDITION_OCCURRENCE c ON d.person_id = c.person_id
LEFT JOIN CONCEPT c1 ON p.race_concept_id = c1.concept_id
LEFT JOIN CONCEPT c2 ON p.gender_concept_id = c2.concept_id
LEFT JOIN CONCEPT cc ON c.condition_concept_id = cc.concept_id
LEFT JOIN CONCEPT dc ON d.drug_concept_id = dc.concept_id;

post condition drugs:
SELECT *
FROM con_drug cd
WHERE
    cd.drug_exposure_start_date > cd.condition_start_date;


In [142]:
query = text("""SELECT p.person_id,
    p.birth_datetime,
    cc.concept_name AS condition_name,
    c.condition_start_date,
    dc.concept_name AS drug_name,
    d.drug_exposure_start_date
FROM DRUG_EXPOSURE d
INNER JOIN PERSON p ON d.person_id = p.person_id
LEFT JOIN CONDITION_OCCURRENCE c ON d.person_id = c.person_id
LEFT JOIN CONCEPT cc ON c.condition_concept_id = cc.concept_id
LEFT JOIN CONCEPT dc ON d.drug_concept_id = dc.concept_id;""")

con_drug_df = pd.read_sql(query, con=engine)
con_drug_df

2024-04-19 18:19:38,594 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 18:19:38,599 INFO sqlalchemy.engine.Engine SELECT p.person_id,
    p.birth_datetime,
    cc.concept_name AS condition_name,
    c.condition_start_date,
    dc.concept_name AS drug_name,
    d.drug_exposure_start_date
FROM DRUG_EXPOSURE d
INNER JOIN PERSON p ON d.person_id = p.person_id
LEFT JOIN CONDITION_OCCURRENCE c ON d.person_id = c.person_id
LEFT JOIN CONCEPT cc ON c.condition_concept_id = cc.concept_id
LEFT JOIN CONCEPT dc ON d.drug_concept_id = dc.concept_id;
2024-04-19 18:19:38,599 INFO sqlalchemy.engine.Engine [generated in 0.01354s] {}
2024-04-19 18:19:38,655 INFO sqlalchemy.engine.Engine ROLLBACK


,person_id,birth_datetime,condition_name,condition_start_date,drug_name,drug_exposure_start_date
0,5675534,1977-03-19,None,2018-09-05,Microencapsulated potassium chloride 20 MEQ Ex...,2018-08-04
1,5675534,1977-03-19,None,2018-08-21,Microencapsulated potassium chloride 20 MEQ Ex...,2018-08-04
2,5675534,1977-03-19,Congestive heart failure,2018-08-04,Microencapsulated potassium chloride 20 MEQ Ex...,2018-08-04
3,5675534,1977-03-19,None,2018-09-05,Microencapsulated potassium chloride 20 MEQ Ex...,2018-08-22
4,5675534,1977-03-19,None,2018-08-21,Microencapsulated potassium chloride 20 MEQ Ex...,2018-08-22
...,...,...,...,...,...,...
104,1372782,1976-06-01,Toxic gastroenteritis,2020-03-13,100 ML metronidazole 5 MG/ML Injection,2020-02-13
105,1372782,1976-06-01,Arteriosclerotic gangrene,2020-03-13,100 ML metronidazole 5 MG/ML Injection,2020-02-13
106,1372782,1976-06-01,Atherosclerosis of artery of lower limb,2020-03-13,100 ML metronidazole 5 MG/ML Injection,2020-02-13
107,1372782,1976-06-01,Peritonitis,2020-03-13,100 ML metronidazole 5 MG/ML Injection,2020-02-13


In [128]:
import pandas
query = text("""SELECT p.person_id,
    p.birth_datetime,
    c2.concept_name as gender,
    c1.concept_name as race,
    cc.concept_name AS condition_name,
    c.condition_start_date,
    c.condition_end_date,
    dc.concept_name AS drug_name,
    d.drug_exposure_start_date,
    d.drug_exposure_end_date
FROM DRUG_EXPOSURE d
INNER JOIN PERSON p ON d.person_id = p.person_id
LEFT JOIN CONDITION_OCCURRENCE c ON d.person_id = c.person_id
LEFT JOIN CONCEPT c1 ON p.race_concept_id = c1.concept_id
LEFT JOIN CONCEPT c2 ON p.gender_concept_id = c2.concept_id
LEFT JOIN CONCEPT cc ON c.condition_concept_id = cc.concept_id
LEFT JOIN CONCEPT dc ON d.drug_concept_id = dc.concept_id;""")

con_drug_df = pd.read_sql(query, con=engine)
con_drug_df

2024-04-19 16:35:12,185 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 16:35:12,186 INFO sqlalchemy.engine.Engine SELECT p.person_id,
    p.birth_datetime,
    c2.concept_name as gender,
    c1.concept_name as race,
    cc.concept_name AS condition_name,
    c.condition_start_date,
    c.condition_end_date,
    dc.concept_name AS drug_name,
    d.drug_exposure_start_date,
    d.drug_exposure_end_date
FROM DRUG_EXPOSURE d
INNER JOIN PERSON p ON d.person_id = p.person_id
LEFT JOIN CONDITION_OCCURRENCE c ON d.person_id = c.person_id
LEFT JOIN CONCEPT c1 ON p.race_concept_id = c1.concept_id
LEFT JOIN CONCEPT c2 ON p.gender_concept_id = c2.concept_id
LEFT JOIN CONCEPT cc ON c.condition_concept_id = cc.concept_id
LEFT JOIN CONCEPT dc ON d.drug_concept_id = dc.concept_id;
2024-04-19 16:35:12,187 INFO sqlalchemy.engine.Engine [generated in 0.00548s] {}
2024-04-19 16:35:12,257 INFO sqlalchemy.engine.Engine ROLLBACK


,person_id,birth_datetime,gender,race,condition_name,condition_start_date,condition_end_date,drug_name,drug_exposure_start_date,drug_exposure_end_date
0,5675534,1977-03-19,male,Black Or African American,None,2018-09-05,None,Microencapsulated potassium chloride 20 MEQ Ex...,2018-08-04,2018-08-04
1,5675534,1977-03-19,male,Black Or African American,None,2018-08-21,None,Microencapsulated potassium chloride 20 MEQ Ex...,2018-08-04,2018-08-04
2,5675534,1977-03-19,male,Black Or African American,Congestive heart failure,2018-08-04,None,Microencapsulated potassium chloride 20 MEQ Ex...,2018-08-04,2018-08-04
3,5675534,1977-03-19,male,Black Or African American,None,2018-09-05,None,Microencapsulated potassium chloride 20 MEQ Ex...,2018-08-22,2018-08-22
4,5675534,1977-03-19,male,Black Or African American,None,2018-08-21,None,Microencapsulated potassium chloride 20 MEQ Ex...,2018-08-22,2018-08-22
...,...,...,...,...,...,...,...,...,...,...
104,1372782,1976-06-01,Female,Asian,Toxic gastroenteritis,2020-03-13,None,100 ML metronidazole 5 MG/ML Injection,2020-02-13,2020-04-09
105,1372782,1976-06-01,Female,Asian,Arteriosclerotic gangrene,2020-03-13,None,100 ML metronidazole 5 MG/ML Injection,2020-02-13,2020-04-09
106,1372782,1976-06-01,Female,Asian,Atherosclerosis of artery of lower limb,2020-03-13,None,100 ML metronidazole 5 MG/ML Injection,2020-02-13,2020-04-09
107,1372782,1976-06-01,Female,Asian,Peritonitis,2020-03-13,None,100 ML metronidazole 5 MG/ML Injection,2020-02-13,2020-04-09


In [117]:
table_name = 'CONDITION_OCCURRENCE'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 16:12:16,753 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 16:12:16,906 INFO sqlalchemy.engine.Engine SELECT * FROM CONDITION_OCCURRENCE


2024-04-19 16:12:16,913 INFO sqlalchemy.engine.Engine [cached since 1.702e+04s ago] {}
2024-04-19 16:12:16,976 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,condition_status_concept_id,stop_reason,provider_id,visit_occurrence_id,visit_detail_id,condition_source_value,condition_source_concept_id,condition_status_source_value
0,66677,5675534,319835,2018-08-04,2018-08-04 10:43:00,None,None,24498375,3680479,None,None,66217400,None,I50.43,35201517,68094332
1,238140,1372782,4043371,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,K52,35201517,68094332
2,248088,1372782,196152,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,K65.8,35201569,68094332
3,276646,1372782,37110250,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,I70.3,45541306,68094332
4,276647,1372782,317577,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,I71.2,45560456,68094332
5,289289,1372782,197596,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,K52,35201517,68094332
6,289385,1372782,319835,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,I50.21,45536428,68094332
7,289387,1372782,193782,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,N19,45541899,68094332
8,301436,1372782,4000609,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,K40,35201492,68094332
9,306327,1372782,4281826,2020-03-13,2020-03-13 21:32:00,None,None,24498375,3680479,None,None,66217400,None,D72.2,45598626,68094332


In [121]:
table_name = 'con_drug'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 16:23:49,039 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 16:23:49,040 INFO sqlalchemy.engine.Engine SELECT * FROM con_drug
2024-04-19 16:23:49,040 INFO sqlalchemy.engine.Engine [cached since 1.845e+04s ago] {}
2024-04-19 16:23:49,070 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,race_concept_id,ethnicity_concept_id,drug_exposure_id,drug_concept_id,drug_exposure_start_date,drug_exposure_end_date,condition_occurrence_id,condition_concept_id,condition_start_date,condition_end_date,condition_name,drug_name
0,5675534,41312805,1977,3,19,66754,826343,3456,36249642,2018-08-04,2018-08-04,675654,319836,2018-09-05,None,None,Microencapsulated potassium chloride 20 MEQ Ex...
1,5675534,41312805,1977,3,19,66754,826343,3456,36249642,2018-08-04,2018-08-04,645454,319837,2018-08-21,None,None,Microencapsulated potassium chloride 20 MEQ Ex...
2,5675534,41312805,1977,3,19,66754,826343,3456,36249642,2018-08-04,2018-08-04,66677,319835,2018-08-04,None,Congestive heart failure,Microencapsulated potassium chloride 20 MEQ Ex...
3,5675534,41312805,1977,3,19,66754,826343,86043,36249642,2018-08-22,2018-08-22,675654,319836,2018-09-05,None,None,Microencapsulated potassium chloride 20 MEQ Ex...
4,5675534,41312805,1977,3,19,66754,826343,86043,36249642,2018-08-22,2018-08-22,645454,319837,2018-08-21,None,None,Microencapsulated potassium chloride 20 MEQ Ex...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,1372782,1995432138,1976,6,1,1099923437,37926810,157016,1707348,2020-02-13,2020-04-09,289289,197596,2020-03-13,None,Toxic gastroenteritis,100 ML metronidazole 5 MG/ML Injection
105,1372782,1995432138,1976,6,1,1099923437,37926810,157016,1707348,2020-02-13,2020-04-09,276647,317577,2020-03-13,None,Arteriosclerotic gangrene,100 ML metronidazole 5 MG/ML Injection
106,1372782,1995432138,1976,6,1,1099923437,37926810,157016,1707348,2020-02-13,2020-04-09,276646,37110250,2020-03-13,None,Atherosclerosis of artery of lower limb,100 ML metronidazole 5 MG/ML Injection
107,1372782,1995432138,1976,6,1,1099923437,37926810,157016,1707348,2020-02-13,2020-04-09,248088,196152,2020-03-13,None,Peritonitis,100 ML metronidazole 5 MG/ML Injection


In [109]:
table_name = 'combined_person_measurement'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 15:59:13,714 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 15:59:13,716 INFO sqlalchemy.engine.Engine SELECT * FROM combined_person_measurement
2024-04-19 15:59:13,716 INFO sqlalchemy.engine.Engine [cached since 3123s ago] {}
2024-04-19 15:59:13,773 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,person_id,gender,birth_datetime,race,measurement_id,measurement_person_id,measurement_concept_name,measurement_datetime,value_as_number,range_low,range_high,unit_source_value
0,5675534,male,1977-03-19,Black Or African American,58776,5675534,Potassium [Moles/volume] in Serum or Plasma,2016-12-31 20:40:00,4.0,3.5,5.1,mmol/L
1,5675534,male,1977-03-19,Black Or African American,74577,5675534,Potassium [Moles/volume] in Serum or Plasma,2018-08-07 02:38:00,4.2,3.5,5.1,mmol/L
2,5675534,male,1977-03-19,Black Or African American,435677,5675534,Potassium [Moles/volume] in Serum or Plasma,2018-08-22 02:27:00,4.4,3.5,5.1,mmol/L
3,5675534,male,1977-03-19,Black Or African American,3475780,5675534,Potassium [Moles/volume] in Serum or Plasma,2018-08-20 03:36:00,3.8,3.5,5.1,mmol/L
4,5675534,male,1977-03-19,Black Or African American,5688646,5675534,Potassium [Moles/volume] in Serum or Plasma,2018-08-04 07:06:00,3.1,3.5,5.1,mmol/L
5,5675534,male,1977-03-19,Black Or African American,6435436,5675534,Potassium [Moles/volume] in Serum or Plasma,2018-08-21 05:59:00,3.8,3.5,5.1,mmol/L


In [101]:
table_name = 'PROCEDURE_OCCURRENCE'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 15:43:19,995 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 15:43:19,997 INFO sqlalchemy.engine.Engine SELECT * FROM PROCEDURE_OCCURRENCE
2024-04-19 15:43:19,997 INFO sqlalchemy.engine.Engine [cached since 172s ago] {}
2024-04-19 15:43:20,037 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_datetime,procedure_end_date,procedure_end_datetime,procedure_type_concept_id,modifier_concept_id,quantity,provider_id,visit_occurrence_id,visit_detail_id,procedure_source_value,procedure_source_concept_id,modifier_source_value
0,11289,1861428,2681195,2020-02-27,2020-02-27 00:00:00,None,None,36429305,0,None,None,8790700,None,0DB18ZX,2181888,No matching concept
1,11443,1861428,2681448,2020-02-27,2020-02-27 00:00:00,None,None,36429305,0,None,None,66217400,None,0DB68ZX,2182141,No matching concept
2,26955,1861428,2681949,2020-02-27,2020-02-27 00:00:00,None,None,36429305,0,None,None,66217400,None,0DBK8ZX,2182642,No matching concept
3,30589,1861428,2667180,2020-03-09,2020-03-09 00:00:00,None,None,36429305,0,None,None,66217400,None,047Q3ZZ,2167873,No matching concept
4,34443,1861428,4237056,2020-03-14,2020-03-14 21:32:00,None,None,38350029,0,None,None,66217400,None,Z78.1,45016441,No matching concept
5,34474,1861428,2681478,2020-02-27,2020-02-27 00:00:00,None,None,36429305,0,None,None,66217400,None,0DB98ZX,2182171,No matching concept
6,38182,1861428,2722801,2020-03-09,2020-03-09 00:00:00,None,None,36429305,0,None,None,8790700,None,B41FYZZ,2223494,No matching concept
7,43885,1861428,2723392,2020-03-11,2020-03-11 00:00:00,None,None,36429305,0,None,None,66217400,None,30233N1,2224085,No matching concept
8,58054,1861428,2684013,2020-03-14,2020-03-14 00:00:00,None,None,36429305,0,None,None,8790700,None,0DJW4ZZ,2184706,No matching concept
9,84354,1861428,2681692,2020-02-27,2020-02-27 00:00:00,None,None,36429305,0,None,None,66217400,None,0DBB8ZX,2182385,No matching concept


In [107]:
table_name = 'procedure_occurrence'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 15:55:57,448 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 15:55:57,449 INFO sqlalchemy.engine.Engine SELECT * FROM procedure_occurrence
2024-04-19 15:55:57,449 INFO sqlalchemy.engine.Engine [cached since 3.01e+04s ago] {}
2024-04-19 15:55:57,466 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_datetime,procedure_end_date,procedure_end_datetime,procedure_type_concept_id,modifier_concept_id,quantity,provider_id,visit_occurrence_id,visit_detail_id,procedure_source_value,procedure_source_concept_id,modifier_source_value
0,11289,1861428,2746520,2020-02-27,2020-02-27,None,None,36429305,0,None,None,8790700,None,0DB18ZX,2746520,No matching concept
1,11443,1861428,2746773,2020-02-27,2020-02-27,None,None,36429305,0,None,None,66217400,None,0DB68ZX,2746773,No matching concept
2,26955,1861428,2747274,2020-02-27,2020-02-27,None,None,36429305,0,None,None,66217400,None,0DBK8ZX,2747274,No matching concept
3,30589,1861428,2732505,2020-03-09,2020-03-09,None,None,36429305,0,None,None,66217400,None,047Q3ZZ,2732505,No matching concept
4,34474,1861428,2746803,2020-02-27,2020-02-27,None,None,36429305,0,None,None,66217400,None,0DB98ZX,2746803,No matching concept
5,38182,1861428,2788126,2020-03-09,2020-03-09,None,None,36429305,0,None,None,8790700,None,B41FYZZ,2788126,No matching concept
6,43885,1861428,2788717,2020-03-11,2020-03-11,None,None,36429305,0,None,None,66217400,None,30233N1,2788717,No matching concept
7,58054,1861428,2749338,2020-03-14,2020-03-14,None,None,36429305,0,None,None,8790700,None,0DJW4ZZ,2749338,No matching concept
8,84354,1861428,2747017,2020-02-27,2020-02-27,None,None,36429305,0,None,None,66217400,None,0DBB8ZX,2747017,No matching concept


In [108]:
table_name = 'combined_procedure_visit'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 15:57:28,836 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 15:57:28,837 INFO sqlalchemy.engine.Engine SELECT * FROM combined_procedure_visit
2024-04-19 15:57:28,837 INFO sqlalchemy.engine.Engine [cached since 653.1s ago] {}
2024-04-19 15:57:28,864 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,visit_occurrence_id,visit_concept_name,visit_start_date,visit_end_date,care_site_id,person_id,procedure_occurrence_id,pocedure_name,procedure_date,procedure_source_value
0,8790700,Outpatient Visit,2016-07-20,2017-09-19,None,1861428,11289,"Excision of Upper Esophagus, Via Natural or Ar...",2020-02-27,0DB18ZX
1,66217400,Emergency Room and Inpatient Visit,2018-06-27,2019-08-17,None,1861428,11443,"Excision of Stomach, Via Natural or Artificial...",2020-02-27,0DB68ZX
2,66217400,Emergency Room and Inpatient Visit,2018-06-27,2019-08-17,None,1861428,26955,"Excision of Ascending Colon, Via Natural or Ar...",2020-02-27,0DBK8ZX
3,66217400,Emergency Room and Inpatient Visit,2018-06-27,2019-08-17,None,1861428,30589,"Dilation of Left Anterior Tibial Artery, Percu...",2020-03-09,047Q3ZZ
4,66217400,Emergency Room and Inpatient Visit,2018-06-27,2019-08-17,None,1861428,34474,"Excision of Duodenum, Via Natural or Artificia...",2020-02-27,0DB98ZX
5,8790700,Outpatient Visit,2016-07-20,2017-09-19,None,1861428,38182,Fluoroscopy of Right Lower Extremity Arteries ...,2020-03-09,B41FYZZ
6,66217400,Emergency Room and Inpatient Visit,2018-06-27,2019-08-17,None,1861428,43885,Transfusion of Nonautologous Red Blood Cells i...,2020-03-11,30233N1
7,8790700,Outpatient Visit,2016-07-20,2017-09-19,None,1861428,58054,"Inspection of Peritoneum, Percutaneous Endosco...",2020-03-14,0DJW4ZZ
8,66217400,Emergency Room and Inpatient Visit,2018-06-27,2019-08-17,None,1861428,84354,"Excision of Ileum, Via Natural or Artificial O...",2020-02-27,0DBB8ZX


In [131]:
table_name = 'con_drug'
query = text(f"SELECT * FROM {table_name} cd WHERE cd.drug_exposure_start_date > cd.condition_start_date;")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-19 16:38:10,782 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 16:38:10,783 INFO sqlalchemy.engine.Engine SELECT * FROM con_drug cd WHERE cd.drug_exposure_start_date > cd.condition_start_date;
2024-04-19 16:38:10,783 INFO sqlalchemy.engine.Engine [cached since 957.4s ago] {}
2024-04-19 16:38:10,843 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,person_id,birth_datetime,gender,race,condition_name,condition_start_date,condition_end_date,drug_name,drug_exposure_start_date,drug_exposure_end_date
0,5675534,1977-03-19,male,Black Or African American,None,2018-08-21,None,Microencapsulated potassium chloride 20 MEQ Ex...,2018-08-22,2018-08-22
1,5675534,1977-03-19,male,Black Or African American,Congestive heart failure,2018-08-04,None,Microencapsulated potassium chloride 20 MEQ Ex...,2018-08-22,2018-08-22
2,1372782,1976-06-01,Female,Asian,Hypervolemia,2020-03-13,None,potassium chloride 20 MEQ Extended Release Ora...,2020-04-26,2020-03-23
3,1372782,1976-06-01,Female,Asian,Lymphocytosis,2020-03-13,None,potassium chloride 20 MEQ Extended Release Ora...,2020-04-26,2020-03-23
4,1372782,1976-06-01,Female,Asian,None,2020-03-13,None,potassium chloride 20 MEQ Extended Release Ora...,2020-04-26,2020-03-23
5,1372782,1976-06-01,Female,Asian,End-stage renal disease,2020-03-13,None,potassium chloride 20 MEQ Extended Release Ora...,2020-04-26,2020-03-23
6,1372782,1976-06-01,Female,Asian,Congestive heart failure,2020-03-13,None,potassium chloride 20 MEQ Extended Release Ora...,2020-04-26,2020-03-23
7,1372782,1976-06-01,Female,Asian,Toxic gastroenteritis,2020-03-13,None,potassium chloride 20 MEQ Extended Release Ora...,2020-04-26,2020-03-23
8,1372782,1976-06-01,Female,Asian,Arteriosclerotic gangrene,2020-03-13,None,potassium chloride 20 MEQ Extended Release Ora...,2020-04-26,2020-03-23
9,1372782,1976-06-01,Female,Asian,Atherosclerosis of artery of lower limb,2020-03-13,None,potassium chloride 20 MEQ Extended Release Ora...,2020-04-26,2020-03-23


In [136]:
table_name = 'PERSON'
query = text(f"""SELECT
    p.person_id, c1.concept_name as measurement_before_drug_name, m1.measurement_datetime AS measurement_before_drug_datetime,
    m1.value_as_number AS measurement_before_drug_value, c2.concept_name as measurement_after_drug_name,
    m2.measurement_datetime AS measurement_after_drug_datetime, m2.value_as_number AS measurement_after_drug_value
FROM
    {table_name} p
JOIN
    CONDITION_OCCURRENCE co ON p.person_id = co.person_id
JOIN
    MEASUREMENT m1 ON p.person_id = m1.person_id AND m1.measurement_datetime > co.condition_start_date
JOIN
    DRUG_EXPOSURE de ON p.person_id = de.person_id AND de.drug_exposure_start_date > co.condition_start_date
JOIN
    MEASUREMENT m2 ON p.person_id = m2.person_id AND m2.measurement_datetime > de.drug_exposure_start_date

LEFT JOIN CONCEPT c1 ON m1.measurement_concept_id = c1.concept_id
LEFT JOIN CONCEPT c2 ON m2.measurement_concept_id = c2.concept_id

WHERE
    co.condition_concept_id = 319835;""")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result


2024-04-19 17:04:53,914 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-19 17:04:53,916 INFO sqlalchemy.engine.Engine SELECT
    p.person_id,
    c1.concept_name as measurement_before_drug_name,
    m1.measurement_datetime AS measurement_before_drug_datetime,
    m1.value_as_number AS measurement_before_drug_value,
    c2.concept_name as measurement_after_drug_name,
    m2.measurement_datetime AS measurement_after_drug_datetime,
    m2.value_as_number AS measurement_after_drug_value
FROM
    PERSON p
JOIN
    CONDITION_OCCURRENCE co ON p.person_id = co.person_id
JOIN
    MEASUREMENT m1 ON p.person_id = m1.person_id AND m1.measurement_datetime > co.condition_start_date
JOIN
    DRUG_EXPOSURE de ON p.person_id = de.person_id AND de.drug_exposure_start_date > co.condition_start_date
JOIN
    MEASUREMENT m2 ON p.person_id = m2.person_id AND m2.measurement_datetime > de.drug_exposure_start_date

LEFT JOIN CONCEPT c1 ON m1.measurement_concept_id = c1.concept_id
LEFT JOIN CONCEPT c2 ON

,person_id,measurement_before_drug_name,measurement_before_drug_datetime,measurement_before_drug_value,measurement_after_drug_name,measurement_after_drug_datetime,measurement_after_drug_value
0,5675534,Potassium [Moles/volume] in Serum or Plasma,2018-08-21 05:59:00,3.8,Potassium [Moles/volume] in Serum or Plasma,2018-08-22 02:27:00,4.4
1,5675534,Potassium [Moles/volume] in Serum or Plasma,2018-08-04 07:06:00,3.1,Potassium [Moles/volume] in Serum or Plasma,2018-08-22 02:27:00,4.4
2,5675534,Potassium [Moles/volume] in Serum or Plasma,2018-08-20 03:36:00,3.8,Potassium [Moles/volume] in Serum or Plasma,2018-08-22 02:27:00,4.4
3,5675534,Potassium [Moles/volume] in Serum or Plasma,2018-08-22 02:27:00,4.4,Potassium [Moles/volume] in Serum or Plasma,2018-08-22 02:27:00,4.4
4,5675534,Potassium [Moles/volume] in Serum or Plasma,2018-08-07 02:38:00,4.2,Potassium [Moles/volume] in Serum or Plasma,2018-08-22 02:27:00,4.4
